In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [4]:
q = """
SELECT 
    *
FROM
    custom.forecasting_hockey_tickets_2526_data_current_reporting
"""

df = FLA_Redshift(**rs_creds).query_warehouse(sql_string = q)

In [5]:
df

,event_date,tier,abbreviation,dow,start_time_tableau,days_out_from_event,is_premier,current_nightly_seats,current_nightly_revenue,current_nightly_atp,...,secondary_num_listings_rolling_3d_avg,secondary_qty_available,secondary_qty_available_rolling_7d_avg,current_logitix_seats,current_logitix_revenue,current_logitix_atp,secondary_pacing_index,secondary_sales_acceleration_rolling_7d_avg,secondary_scarcity_index,processed_date
0,2026-01-31,C,WPG,Sat,4 PM,2,1,1883,207084.20,109.975677,...,216.0,1110,1110.0,2003,225217.90239,112.440291,1.059788,1.436945,0.954764,2026-01-29 06:18:20
1,2026-02-02,E,BUF,Mon,7 PM,4,0,3471,221433.43,63.795284,...,256.0,1584,1584.0,1430,76575.07224,53.549002,1.232759,2.142708,0.778257,2026-01-29 06:18:20
2,2026-02-04,B,BOS,Wed,7 PM,6,1,955,190149.49,199.109414,...,241.0,1407,1407.0,913,148167.77315,162.286718,0.573492,1.948421,0.407599,2026-01-29 06:18:20
3,2026-02-26,B,TOR,Thu,7 PM,28,1,910,175625.15,192.994670,...,257.0,1538,1538.0,697,107094.39129,153.650490,0.929333,1.127439,0.604248,2026-01-29 06:18:20
4,2026-02-27,C,BUF,Fri,7 PM,29,1,1768,173089.56,97.901335,...,266.0,1623,1623.0,940,86409.73577,91.925251,1.358382,1.140299,0.836957,2026-01-29 06:18:20
5,2026-03-10,D,DET,Tue,7 PM,40,0,1551,188327.72,121.423417,...,264.0,1622,1622.0,520,50795.41605,97.683492,1.306533,0.994393,0.805507,2026-01-29 06:18:20
6,2026-03-12,D,CBJ,Thu,7 PM,42,0,1175,135938.28,115.692153,...,274.0,1680,1680.0,361,27509.41999,76.203379,0.937662,1.220158,0.558132,2026-01-29 06:18:20
7,2026-03-24,E,SEA,Tue,7 PM,54,0,2106,148824.20,70.666762,...,275.0,1661,1661.0,280,20665.17805,73.804207,1.450777,1.259299,0.873436,2026-01-29 06:18:20
8,2026-03-26,D,MIN,Thu,7 PM,56,0,1380,157545.75,114.163587,...,268.0,1665,1665.0,575,46032.67655,80.056829,2.220077,1.418449,1.333380,2026-01-29 06:18:20
9,2026-03-31,D,OTT,Tue,7 PM,61,0,794,111805.05,140.812406,...,277.0,1703,1703.0,261,20176.22293,77.303536,1.082988,1.749317,0.635929,2026-01-29 06:18:20
